# Analyze Detection Results - Compare Detections vs Real Calls

This notebook helps you:
1. Load detection results from CSV files
2. Extract audio segments from detected time ranges
3. Visualize mel-spectrograms of detections
4. Compare side-by-side: **Detected samples vs Real primate calls**
5. Identify false positives for hard negative mining

**Goal**: Understand what the model is detecting and whether they are correct

## Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install -q librosa soundfile matplotlib pandas


In [ ]:
# Import modules
import sys
sys.path.append('src')  # Adjust to your path

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pandas as pd
import os
from IPython.display import Audio, display

import config
import data_loader
import preprocessing

config.print_config_summary()

## Load Detection Results

In [ ]:
# List all detection CSV files
detection_dir = config.DETECTION_OUTPUT_DIR

detection_files = [f for f in os.listdir(detection_dir) if f.endswith('_detections.csv')]

print(f"Found {len(detection_files)} detection files:")
for i, f in enumerate(detection_files):
    print(f"  {i}: {f}")

In [ ]:
# Choose which detection file to analyze
file_idx = 0  # Change this to select different file

detection_csv = os.path.join(detection_dir, detection_files[file_idx])
detections_df = pd.read_csv(detection_csv)

# Get corresponding audio file name
audio_filename = detection_files[file_idx].replace('_detections.csv', '.wav')
audio_path = os.path.join(config.LONG_AUDIO_ROOT, audio_filename)

print(f"\nAnalyzing: {audio_filename}")
print(f"Total detections: {len(detections_df)}")
print(f"\nDetection summary:")
print(detections_df.head(10))

if len(detections_df) > 0:
    print(f"\nBy species:")
    print(detections_df['species'].value_counts())
    
    print(f"\nConfidence statistics:")
    print(detections_df.groupby('species')['confidence'].describe())

## Filter Uncertain Detections (Potential False Positives)

In [ ]:
# Focus on medium-confidence detections (0.5 - 0.85)
# These are most likely to be false positives

min_conf = 0.5
max_conf = 0.85

uncertain_df = detections_df[
    (detections_df['confidence'] >= min_conf) & 
    (detections_df['confidence'] <= max_conf)
].copy()

uncertain_df = uncertain_df.sort_values('confidence', ascending=False).reset_index(drop=True)

print(f"Uncertain detections ({min_conf}-{max_conf} confidence): {len(uncertain_df)}")
print(f"\nBy species:")
if len(uncertain_df) > 0:
    print(uncertain_df['species'].value_counts())
    print(f"\nTop 10 uncertain detections:")
    print(uncertain_df.head(10))
else:
    print("No uncertain detections found. Try adjusting confidence range.")

## Load Real Primate Calls (for comparison)

In [ ]:
# Load real primate samples
print("Loading real primate call samples...\n")

species_data = data_loader.load_species_data()

# Store a few examples from each species
real_samples = {}

for species_name, audio_list in species_data.items():
    # Take first 3 samples as references
    real_samples[species_name] = [
        {
            'audio': audio,
            'filepath': filepath,
            'spec': preprocessing.audio_to_melspectrogram(audio)
        }
        for audio, filepath in audio_list[:3]
    ]
    
    print(f"{species_name}: Loaded {len(real_samples[species_name])} reference samples")

print("\nReal samples ready for comparison!")

## Visualization Function

In [ ]:
def compare_detection_with_real(detection_row, audio_path, real_samples_dict, play_audio=True):
    """
    Compare a detection with real primate calls
    
    Shows:
    - Left: Detected segment from long audio
    - Right: Real primate call of the same species
    """
    species = detection_row['species']
    start_time = detection_row['start_time']
    end_time = detection_row['end_time']
    confidence = detection_row['confidence']
    
    # Load long audio and extract segment
    print(f"Loading segment from {os.path.basename(audio_path)}...")
    full_audio, sr = librosa.load(audio_path, sr=config.SAMPLE_RATE)
    
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    detected_audio = full_audio[start_sample:end_sample]
    
    # Compute detected spectrogram
    detected_spec = preprocessing.audio_to_melspectrogram(detected_audio, sr)
    
    # Get a real sample of the same species
    if species in real_samples_dict and len(real_samples_dict[species]) > 0:
        real_sample = real_samples_dict[species][0]  # Take first reference
        real_audio = real_sample['audio']
        real_spec = real_sample['spec']
        real_filepath = real_sample['filepath']
    else:
        print(f"⚠️  No real samples available for {species}")
        real_audio = None
        real_spec = None
        real_filepath = None
    
    # Create figure
    if real_spec is not None:
        fig, axes = plt.subplots(2, 2, figsize=(18, 10))
    else:
        fig, axes = plt.subplots(2, 1, figsize=(18, 10))
        axes = axes.reshape(2, 1)
    
    fig.suptitle(
        f"DETECTION vs REAL CALL - {species}\n" +
        f"Detection: {start_time:.1f}-{end_time:.1f}s, Confidence: {confidence:.3f}",
        fontsize=16, fontweight='bold'
    )
    
    # ===== LEFT COLUMN: DETECTED SEGMENT =====
    
    # Detected waveform
    times = np.arange(len(detected_audio)) / sr
    axes[0, 0].plot(times, detected_audio, linewidth=0.5, color='red', alpha=0.8)
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Amplitude')
    axes[0, 0].set_title(f'DETECTED - Waveform\n(from long audio at {start_time:.1f}s)', 
                         fontsize=12, fontweight='bold', color='red')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Detected spectrogram
    img1 = librosa.display.specshow(
        detected_spec,
        x_axis='time',
        y_axis='mel',
        sr=sr,
        hop_length=config.HOP_LENGTH,
        fmin=config.FMIN,
        fmax=config.FMAX,
        ax=axes[1, 0],
        cmap='viridis'
    )
    axes[1, 0].set_title(f'DETECTED - Mel-Spectrogram', 
                         fontsize=12, fontweight='bold', color='red')
    fig.colorbar(img1, ax=axes[1, 0], format='%+2.0f dB')
    
    
    if real_spec is not None:
        # Real waveform
        times_real = np.arange(len(real_audio)) / sr
        axes[0, 1].plot(times_real, real_audio, linewidth=0.5, color='green', alpha=0.8)
        axes[0, 1].set_xlabel('Time (s)')
        axes[0, 1].set_ylabel('Amplitude')
        axes[0, 1].set_title(f'REAL {species} - Waveform\n(training sample)', 
                            fontsize=12, fontweight='bold', color='green')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Real spectrogram
        img2 = librosa.display.specshow(
            real_spec,
            x_axis='time',
            y_axis='mel',
            sr=sr,
            hop_length=config.HOP_LENGTH,
            fmin=config.FMIN,
            fmax=config.FMAX,
            ax=axes[1, 1],
            cmap='viridis'
        )
        axes[1, 1].set_title(f'REAL {species} - Mel-Spectrogram', 
                            fontsize=12, fontweight='bold', color='green')
        fig.colorbar(img2, ax=axes[1, 1], format='%+2.0f dB')
    
    plt.tight_layout()
    plt.show()
    
    # Audio playback
    if play_audio:
        print("\n🔊 Audio Playback:")
        print("\nDETECTED segment:")
        display(Audio(detected_audio, rate=sr))
        
        if real_audio is not None:
            print(f"\nREAL {species} call (for comparison):")
            display(Audio(real_audio, rate=sr))
    

## Analyze Uncertain Detections One by One

In [ ]:
# Analyze first 10 uncertain detections
n_to_analyze = min(10, len(uncertain_df))

print(f"Analyzing {n_to_analyze} uncertain detections...\n")

for i in range(n_to_analyze):

    print(f"# DETECTION {i+1}/{n_to_analyze}")

    
    detection = uncertain_df.iloc[i]
    
    compare_detection_with_real(
        detection,
        audio_path,
        real_samples,
        play_audio=True
    )
    

## Examine Specific Detection

In [ ]:
# Choose a specific detection to examine in detail
detection_idx = 0  # Change this number

if len(uncertain_df) > detection_idx:
    detection = uncertain_df.iloc[detection_idx]
    
    
    compare_detection_with_real(
        detection,
        audio_path,
        real_samples,
        play_audio=True
    )
else:
    print(f"Index {detection_idx} out of range (total: {len(uncertain_df)})")

## Compare Multiple Real Samples with One Detection

In [ ]:
def compare_detection_with_multiple_real(detection_row, audio_path, real_samples_dict):
    """
    Show one detection alongside multiple real samples
    """
    species = detection_row['species']
    start_time = detection_row['start_time']
    end_time = detection_row['end_time']
    confidence = detection_row['confidence']
    
    # Load detected segment
    full_audio, sr = librosa.load(audio_path, sr=config.SAMPLE_RATE)
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    detected_audio = full_audio[start_sample:end_sample]
    detected_spec = preprocessing.audio_to_melspectrogram(detected_audio, sr)
    
    # Get real samples
    if species not in real_samples_dict:
        print(f"No real samples for {species}")
        return
    
    real_samples_list = real_samples_dict[species][:3]  # Take up to 3
    
    # Create figure
    n_real = len(real_samples_list)
    fig, axes = plt.subplots(2, n_real + 1, figsize=(6*(n_real+1), 10))
    
    fig.suptitle(
        f"Detection vs Multiple Real {species} Calls\n" +
        f"Confidence: {confidence:.3f}",
        fontsize=16, fontweight='bold'
    )
    
    # Column 0: Detected
    times = np.arange(len(detected_audio)) / sr
    axes[0, 0].plot(times, detected_audio, linewidth=0.5, color='red')
    axes[0, 0].set_title('DETECTED', fontweight='bold', color='red', fontsize=12)
    axes[0, 0].grid(True, alpha=0.3)
    
    librosa.display.specshow(
        detected_spec, x_axis='time', y_axis='mel',
        sr=sr, hop_length=config.HOP_LENGTH,
        fmin=config.FMIN, fmax=config.FMAX,
        ax=axes[1, 0], cmap='viridis'
    )
    axes[1, 0].set_title(f'at {start_time:.1f}s', fontsize=10)
    
    # Columns 1+: Real samples
    for i, sample in enumerate(real_samples_list, 1):
        real_audio = sample['audio']
        real_spec = sample['spec']
        
        times_real = np.arange(len(real_audio)) / sr
        axes[0, i].plot(times_real, real_audio, linewidth=0.5, color='green')
        axes[0, i].set_title(f'REAL #{i}', fontweight='bold', color='green', fontsize=12)
        axes[0, i].grid(True, alpha=0.3)
        
        librosa.display.specshow(
            real_spec, x_axis='time', y_axis='mel',
            sr=sr, hop_length=config.HOP_LENGTH,
            fmin=config.FMIN, fmax=config.FMAX,
            ax=axes[1, i], cmap='viridis'
        )
        axes[1, i].set_title(f'{os.path.basename(sample["filepath"])[:20]}...', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
# Example: Compare one detection with 3 real samples
if len(uncertain_df) > 0:
    detection_idx = 0  # Change this
    
    compare_detection_with_multiple_real(
        uncertain_df.iloc[detection_idx],
        audio_path,
        real_samples
    )
else:
    print("No uncertain detections to analyze")

## Batch Analysis: Show All Uncertain Detections (Spectrograms Only)

In [ ]:
# Quick overview of all uncertain detections
n_to_show = min(20, len(uncertain_df))

if n_to_show > 0:
    print(f"Showing spectrograms of {n_to_show} uncertain detections...\n")
    
    # Load long audio once
    full_audio, sr = librosa.load(audio_path, sr=config.SAMPLE_RATE)
    
    # Create grid
    n_cols = 5
    n_rows = int(np.ceil(n_to_show / n_cols))
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4*n_rows))
    axes = axes.flatten()
    
    for i in range(n_to_show):
        detection = uncertain_df.iloc[i]
        
        # Extract segment
        start_sample = int(detection['start_time'] * sr)
        end_sample = int(detection['end_time'] * sr)
        segment = full_audio[start_sample:end_sample]
        
        # Compute spectrogram
        spec = preprocessing.audio_to_melspectrogram(segment, sr)
        
        # Plot
        librosa.display.specshow(
            spec,
            x_axis='time',
            y_axis='mel',
            sr=sr,
            hop_length=config.HOP_LENGTH,
            fmin=config.FMIN,
            fmax=config.FMAX,
            ax=axes[i],
            cmap='viridis'
        )
        
        axes[i].set_title(
            f"#{i}: {detection['species']}\n" +
            f"{detection['start_time']:.1f}s, conf={detection['confidence']:.2f}",
            fontsize=9
        )
    
    # Hide empty subplots
    for i in range(n_to_show, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(
        f"All Uncertain Detections - {os.path.basename(audio_path)}",
        fontsize=16,
        fontweight='bold'
    )
    
    plt.tight_layout()
    plt.show()
    
    print("\nLook for patterns:")
    print("  - Do some spectrograms look very different from real calls?")
    print("  - Are there common visual features in false positives?")
    print("  - Do bird calls have a distinctive pattern?")
else:
    print("No uncertain detections to show")